### 1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

In [1]:
import pandas as pd
import chardet

In [2]:
with open('ml-100k/u.item','rb') as read_file:
    temp_container = bytes()
    for i, line in enumerate(read_file):
        temp_container += line
        if i > 100: break
    charmap = bytes(temp_container)
    print(chardet.detect(charmap)['encoding'])

ascii


In [3]:
col_ns = ['user_id','item_id','rating','timestamp']
df_ratings = pd.read_csv('ml-100k/u.data', sep='\t', encoding='ascii', names=col_ns)
df_ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
col_names = '''movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western'''

col_n = []
for w in col_names.split('|'):
    col_n.append(w.strip())

In [5]:
df_movies = pd.read_csv('ml-100k/u.item', sep='|', encoding='ascii', encoding_errors='ignore', names=col_n)
df_movies.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### 2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок

In [6]:
df_ratings.groupby('user_id')['timestamp'].count().sort_values(ascending=False).reset_index().iloc[0][0]

405

### 3. Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь

In [7]:
df_ratings_405 = df_ratings.query('user_id == 405')
df_ratings_405

,user_id,item_id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447
...,...,...,...,...
98956,405,375,1,885546835
98978,405,445,4,885548435
99148,405,1246,1,885547735
99465,405,196,1,885546112


### 4. Добавьте к датафрейму из задания 3 столбцы:
- По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет
- столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей

In [8]:
#Простите, я увлекся записывая все в одну строку
df_ratings_405 = pd.merge(
    df_ratings_405, df_movies.loc[df_movies['movie id'].isin(df_ratings['item_id'])].drop(col_n[1:5], axis=1),
    how='inner', left_on='item_id',right_on='movie id').drop('movie id', axis=1)

In [9]:
df_405 = pd.merge(
    df_ratings_405,
    df_ratings.groupby('item_id').agg(
    mean = ('rating','mean'),
    count = ('rating','count')).reset_index(),
    how='left', on='item_id')

df_405.head()

,user_id,item_id,rating,timestamp,unknown,Action,Adventure,Animation,Children's,Comedy,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,mean,count
0,405,56,4,885544911,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.060914,394
1,405,592,1,885548670,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,3.333333,9
2,405,1582,1,885548670,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.000000,1
3,405,171,1,885549544,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,3.876923,65
4,405,580,1,885547447,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,3.375000,32


### 5. Сформируйте X_train, X_test, y_train, y_test

In [10]:
from sklearn.model_selection import train_test_split

# А что мы предсказываем то? Ну пусть будет оценки этого 405
X_train, X_test, y_train, y_test = train_test_split(
    df_405.drop(['user_id','item_id','rating', 'timestamp'],axis=1),
    df_405['rating'],
    test_size=0.3
    )

### 6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

In [11]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lr = LinearDiscriminantAnalysis()
lr.fit(X_train,y_train)
y_predicted = lr.predict(X_test)

### 7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [12]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predicted)

0.6306306306306306

### 8. Загрузить данные в spark

In [13]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [14]:
spdf_ratings = spark.read.csv('ml-100k/u.data', header=False,inferSchema=True, sep='\t')
spdf_movies = spark.read.csv('ml-100k/u.item', header=False, inferSchema=True, sep='|')

In [15]:
for i, old_col in enumerate(spdf_ratings.columns):
    spdf_ratings = spdf_ratings.withColumnRenamed(old_col, col_ns[i])

In [16]:
spdf_ratings.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [17]:
for i, old_col in enumerate(spdf_movies.columns):
    spdf_movies = spdf_movies.withColumnRenamed(old_col, col_n[i])

In [18]:
spdf_movies.show(5)

+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie id|      movie title|release date|video release date|            IMDb URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|              n

### 9. Средствами спарка вывести среднюю оценку для каждого фильма

In [19]:
spdf_ratings.groupBy('item_id').mean('rating').show(5)

+-------+------------------+
|item_id|       avg(rating)|
+-------+------------------+
|    496| 4.121212121212121|
|    471|3.6108597285067874|
|    463| 3.859154929577465|
|    148|          3.203125|
|   1342|               2.5|
+-------+------------------+
only showing top 5 rows



### 10. Посчитайте средствами спарка среднюю оценку для каждого жанра

In [20]:
spdf_movies_cl = spdf_movies
for col in col_n[1:5]:
    spdf_movies_cl = spdf_movies_cl.drop(col)

In [21]:
spdf_joined = spdf_ratings.join(spdf_movies_cl, spdf_ratings['item_id'] == spdf_movies['movie id'], 'inner')
spdf_joined.show(5)

+-------+-------+------+---------+--------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|user_id|item_id|rating|timestamp|movie id|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+-------+-------+------+---------+--------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|    196|    242|     3|881250949|     242|      0|     0|        0|        0|         0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|    186|    302|     3|891717742|     302|      0|     0|        0|        0|         0|     0|    1|          0|    0|      0|        1|     0|      0|      1|      0|     0|       1|  0|      0|
|     22| 

In [22]:
ratings = {}
for col in spdf_joined.columns[5:]:
    ratings[col] = spdf_joined.where(spdf_joined[col] == 1).agg({'rating':'mean'}).collect()[0][0]
print(ratings)

{'unknown': 3.2, 'Action': 3.480245417953027, 'Adventure': 3.503526503308369, 'Animation': 3.5766990291262135, "Children's": 3.3532442216652742, 'Comedy': 3.3940734781442745, 'Crime': 3.6322780881440098, 'Documentary': 3.6728232189973613, 'Drama': 3.6873793708484772, 'Fantasy': 3.2152366863905324, 'Film-Noir': 3.9215233698788228, 'Horror': 3.2903893172841827, 'Musical': 3.521396851029471, 'Mystery': 3.63813155386082, 'Romance': 3.621704948358255, 'Sci-Fi': 3.5607227022780834, 'Thriller': 3.5090069495245064, 'War': 3.815811874866993, 'Western': 3.6132686084142396}


### 11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)

In [23]:
# Сделаю и так и так
top5 = spdf_joined.groupBy('movie id').agg({'rating':'count'}).sort('count(rating)', ascending=False).limit(5)

In [24]:
top5.show()

+--------+-------------+
|movie id|count(rating)|
+--------+-------------+
|      50|          583|
|     258|          509|
|     100|          508|
|     181|          507|
|     294|          485|
+--------+-------------+



In [25]:
lose5 = spdf_joined.groupBy('movie id').agg({'rating':'mean'}).sort('avg(rating)', ascending=True).limit(5)

In [26]:
lose5.show()

+--------+-----------+
|movie id|avg(rating)|
+--------+-----------+
|    1339|        1.0|
|    1580|        1.0|
|    1618|        1.0|
|     858|        1.0|
|    1352|        1.0|
+--------+-----------+

